<a href="https://colab.research.google.com/github/trashchenkov/gigachat_tutorials/blob/main/RAG_%D0%BF%D0%BE_%D0%B2%D0%B8%D0%B4%D0%B5%D0%BE%D1%80%D0%BE%D0%BB%D0%B8%D0%BA%D0%B0%D0%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG по видеороликам
В этом туториале мы создадим тематическую вопросно-ответную систему, которая будет использовать видеоролики из [плейлиста по GigaChat](https://youtube.com/playlist?list=PLtawsUZrloMcft-Aq05bT4gpmJgkWHpB0&si=6IGwgX0Tnn2i4DPS) в качестве базы знаний.

Это будет несколько более сложный проект, чем примеры RAG, которые мы рассматривали в прошлых туториалах, поскольку здесь система не только будет генерировать ответы с помощью большой языковой модели, но и давать возможность посмотреть видеоролики с тех фрагментов, на которых строится сгенерированный ответ.

Для того, чтобы реализовать такую систему, нам придется выполнить ряд этапов, которые мы разобъем на две части:

1. **Преобразование речи из видео в текст**
  - скачать видеоролики для распознавания речи;
  - извлечь аудио из видео;
  - распознать речь, выполнив Speech2Text преобразование.
2. **Создание RAG пайплайна**
  - сформировать фрагменты текстов, чтобы в метаданных содержалась ссылка на ролик и таймкод;
  - создать векторную базу данных из фрагментов;
  - составить пайплайн с использованием инструментов GigaChain;
  - развернуть вопросно-ответную систему в виде приложения на Hugging Face.

Обратите внимание, что в первой части нам нужно будет работать в режиме GPU.

## Преобразование речи из видео в текст (режим GPU)
### Установка библиотек

Для этой части туториала нам понадобятся следующие библиотеки:
- `pytube` - позволяет скачивать видео с ютуб, а также получать некоторую метаинформацию о видео;
- `openai-whisper` - библиотека для работы с линейкой моделей для распознавания речи от OpenAI.

Если будете выполнять не в среде Google Colab, то еще понадобится установка `moviepy`, который в Colab уже установлен.


In [1]:
!pip install pytube openai-whisper --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 40.7 MB/s eta 0:00:00


### Скачивание видеороликов
Используем класс `Playlist` из библиотеки `pytube` для получения списка роликов из плейлиста.

Обратите внимание, что при скачивании файлов из названий убирается часть пунктуации.

In [ ]:
from pytube import Playlist

def download_youtube_playlist(playlist_url, download_path='.'):
    try:
        playlist = Playlist(playlist_url)
        print(f'Скачивание плейлиста: {playlist.title}')
        for video in playlist.videos:
            print(f'Скачивание видео: {video.title}')
            video.streams.get_highest_resolution().download(output_path=download_path)
            print(f'Видео {video.title} скачано успешно.')
        print('Все видео из плейлиста скачаны.')
    except Exception as e:
        print(f'Произошла ошибка: {e}')

# Пример использования
playlist_url = 'https://www.youtube.com/playlist?list=PLtawsUZrloMcft-Aq05bT4gpmJgkWHpB0'

download_youtube_playlist(playlist_url)


Скачивание плейлиста: GigaChat
Скачивание видео: Как начать работать с GigaChat API? Подробный туториал на Python
Видео Как начать работать с GigaChat API? Подробный туториал на Python скачано успешно.
Скачивание видео: Как общаться с GigaChat API с помощью Gigachain? (Туториал)
Видео Как общаться с GigaChat API с помощью Gigachain? (Туториал) скачано успешно.
Скачивание видео: Как подключить GigaChat API к Telegram-боту? Подробное руководство на Python
Видео Как подключить GigaChat API к Telegram-боту? Подробное руководство на Python скачано успешно.
Скачивание видео: Как сделать агентов на основе GigaChat? Подробный туториал на Python
Видео Как сделать агентов на основе GigaChat? Подробный туториал на Python скачано успешно.
Скачивание видео: Реализация RAG на основе GigaChat. Как искать и генерировать ответы по базе знаний?
Видео Реализация RAG на основе GigaChat. Как искать и генерировать ответы по базе знаний? скачано успешно.
Скачивание видео: GigaChain. Как загрузить документы, 

### Извлечение аудио из видео

На вход моделей Whisper можно подавать практически любые форматы аудиофайлов.

Я экспериментировал с WAV и MP3, в итоге остановился на WAV. При создании MP3 и затем при распознавании уходит больше времени на обработку из-за компрессии/декомпрессии. С WAV работа по распознованию речи идет несколько быстрее, но зато файлы WAV занимают значительно больше места. Размер WAV файла в разы, а то и на порядок превышает размер видеофайла, из которого мы извлекаем аудио.

In [3]:

from moviepy.editor import VideoFileClip
from glob import glob

videos = glob('*.mp4')

audios = []
for video in videos:
  vid = VideoFileClip(video)
  audio = video[:-3] + 'wav'
  vid.audio.write_audiofile(audio)
  audios.append(audio)




MoviePy - Writing audio in Как пользоваться GPT-4 и Claude 3 бесплатно shorts.wav


MoviePy - Done.


### Распознавание речи в текст
Для обеспечения лучшего качества распознавания используем модель "large" (около 3 ГБ).

Результатом работы модели является словарь, имеющий следующую структуру:

```json
{
    "text": "Здесь находится целиком распознанный текст. Делее текст делится на сегменты, а секместы на отдельные слова.",
    "segments": [
        {
            "id": 0,
            "seek": 0,
            "start": 0.0,
            "end": 10.0,
            "text": "Здесь находится целиком распознанный текст.",
            "tokens": [токены (числа, обозначающие обнаруженные фонемы)],
            "temperature": 0.0,
            "avg_logprob": -0.06037478833585172,
            "compression_ratio": 1.5186567164179106,
            "no_speech_prob": 0.0030271916184574366,
            "words": [отдельные слова в виде словарей (слово, начало, конец и вероятность)]
        },
      
    ],
    "language": "en"
}

```
Каждый сегмент в segments включает следующие ключи:

  - id - уникальный идентификатор сегмента;
  - seek - позиция в аудиофайле, с которой начинается сегмент;
  - start - начало сегмента (в секундах);
  - end - конец сегмента (в секундах);
  - text - распознанный текст для данного сегмента;
  - tokens - токены, представляющие распознанный текст;
  - temperature - параметр, влияющий на разнообразие предсказаний;
  - avg_logprob - средная логарифмическая вероятность предсказания;
  - compression_ratio - коэффициент сжатия для предсказанного текста;
  - no_speech_prob - вероятность того, что в сегменте нет речи;
  - words - распознанные слова по отдельности с указанием времени начала слова, окончания и вероятностью.



Полученные словари сохраним в json-файлы, чтобы потом уже в CPU среде сделать из них фрагменты текста, которые будут использоваться в RAG. Файлы скачаем, чтобы не потерять результаты распознавания.

Процесс распознования является ресурсоемким. Для обработки [плейлиста по GigaChat](https://youtube.com/playlist?list=PLtawsUZrloMcft-Aq05bT4gpmJgkWHpB0&si=6IGwgX0Tnn2i4DPS) (всего более 5 часов материала) Google Colab в режиме GPU потребовалось около 1,5 часов.

In [5]:
import whisper
import json
from google.colab import files

model = whisper.load_model("large")
for audio in audios:
  result = model.transcribe(audio, word_timestamps=True)
  file_name = audio[:-3] + "json"
  with open(file_name, "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=4)
  files.download(file_name)


100%|█████████████████████████████████████| 2.88G/2.88G [01:02<00:00, 49.6MiB/s]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Демонстрация процесса на примере одного короткого видео

Скачаем мой Shorts ["Как пользоваться GPT-4 и Claude 3 бесплатно?"](https://youtube.com/shorts/Y-ZGLzgVBpA) и запустим ячейки кода, расположенные выше, чтобы извлечь аудио и распознать речь.

In [2]:
from pytube import YouTube
yt = YouTube('https://youtube.com/shorts/Y-ZGLzgVBpA')
yt.streams.get_highest_resolution().download(output_path='.')

'/content/./Как пользоваться GPT-4 и Claude 3 бесплатно shorts.mp4'

Оказывается с помощью whisper можно не только распознать речь, но и сделать перевод на английский.

In [ ]:
result1 = model.transcribe(audio, word_timestamps=True)

MoviePy - Writing audio in audio.mp3


MoviePy - Done.


## Создание RAG пайплайна

Мы уже создавали фрагменты текстов в прошлых туториалах, чтобы их векторизовать и поместить в векторное хранилище. Для этого использовались готовые инструменты из GigaChain (загрузчики и сплиттеры). Поскольку в этот раз мы оперируем нестандартными данными, а также система в итоге должна не только давать ответ на вопрос, но и снабжать пользователя указаниями на место в видео, то придется написать код для подготовки текстов самостоятельно.

### Установка библиотек

В этой части туториала можно работать в режиме CPU.

Для построения пайплайна нам понадобятся следующие библиотеки:

- `gigachain` - наш основной инструмент, который позволяет взаимодействовать с языковой моделью и собирать паплайн воедино;
- `gigachain-community` - содержит в себе основные интеграции с загрузчиками, векторными базами данных и другими компонентами (ранее эа библиотека подтягивалась автоматически при загрузке `gigachain`, с недавних пор нужно загружать отдельно);
- `sentence-transformers` - для взаимодействия с моделями, размещенными на Hugging Face (в нашем случае речь про эмбеддинговую модель);
- `faiss-cpu` - для работы с векторным хранилищем;
- `pytube` - для получения метаданных о видеороликах.


In [1]:
!pip install gigachain gigachain-community sentence-transformers faiss-cpu pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 995.3/995.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 11.2 MB/s eta 0:00:00
  Using cached nvidia_cu

### Подготовка метаданных

Для нашей RAG системы понадобится не только сам текст, но и метаданные (ссылки на ролики, таймкоды для каждого фрагмента).

Создадим словарь, в катором названия роликов будут ключами, а значения - ссылками. Это позволит потом сделать метаданные для каждого фрагмента. Отличительным моментом возвращаемых объектом класса `Playlist` ссылок является то, что они идут в форме для встраивания. Это позволит их легко разместить в пользовательском интерфейсе RAG приложения.

Регулярное выражение пришлось применить, поскольку в назывании видео могут встречаться знаки пунктуации, которых мы не встретим в названиях файлов по этим роликам (видео, аудио и json).

In [2]:
import re
from pytube import Playlist

playlist_url = 'https://www.youtube.com/playlist?list=PLtawsUZrloMcft-Aq05bT4gpmJgkWHpB0'
video_urls = {}
playlist = Playlist(playlist_url)
for v in playlist.videos:
  title = re.sub(r'[^\w\s\-\(\)]', '', v.title)

  video_urls[title] = v.embed_url

### Сборка фрагментов

Обычно мы брали тексты целиком и, используя сплиттер из инструментов GigaChain, нарезали фрагменты. В этот раз мы поступим принципиально иначе. Нам необходимо, чтобы у каждого фрагмента был свой таймкод. Информация о таймкодах содержится в сегментах json-файлов, полученных при распознавании. Сами по себе сегменты слишком короткие, чтобы служить контекстом для RAG, поэтому мы будем брать несколько сегментов и соединять их в один фрагмент. Будем ориентироваться примерно на 1000 знаков в одном фрагменте. Фрагменты будут иметь перекрытия в 3 сегмента.

Далее фрагменты необходимо "упаковать" в класс `Document` из библиотеки GigaChain, поместив туда не только текст, но и метаданные: ссылку на ролик, таймкод начала фрагмента.


При сопоставлении названий роликов и названий json-файлов столкнулся с проблемой, что в json-файлах буква й превращается в 2 символа. Чтобы это исправить и соотнести, в конечном счете, фрагмент и ссылку на ролик, пришлось применять функцию `normalize()` модуля `unicodedata` для превращения двух символов в нормальную букву й.

Объекты класса `Document` собираем в список, чтобы затем провести векторизацию.

In [3]:
import json
from langchain_core.documents import Document
from unicodedata import normalize
from glob import glob


def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

def create_chunks_with_overlap(segments, chunk_size=500, overlap_segments=2):
    chunks = []
    current_chunk = {
        "start_time": round(segments[0]['start']),
        "text": "",
        "segments": []
    }
    current_length = 0

    for segment in segments:
        segment_length = len(segment['text'])
        if current_length + segment_length <= chunk_size:
            current_chunk['text'] += segment['text'] + " "
            current_chunk['segments'].append(segment)
            current_length += segment_length
        else:
            chunks.append(current_chunk)
            overlap_text = "".join([s['text'] for s in current_chunk['segments'][-overlap_segments:]])
            new_segments = current_chunk['segments'][-overlap_segments:]
            current_chunk = {
                "start_time": round(new_segments[0]['start']),
                "text": overlap_text + segment['text'] + " ",
                "segments": new_segments + [segment]
            }
            current_length = len(overlap_text) + segment_length

    if current_chunk['text']:
        chunks.append(current_chunk)

    return chunks

json_files = glob('*.json')
docs = []
for f in json_files:
    nf = normalize('NFC', f)
    link = video_urls[nf[:-5]]
    data = load_json(f)
    segments = data['segments']
    chunks = create_chunks_with_overlap(segments, chunk_size=1000, overlap_segments=3)
    for chunk in chunks:
        metadata = {"link": link, 'time': chunk['start_time']}
        docs.append(Document(page_content=chunk['text'], metadata=metadata))






In [4]:
d = docs[0]

In [5]:
d.metadata

{'link': 'https://www.youtube.com/embed/DfT_xXDQyCQ', 'time': 0}

Получили 228 фрагментов из 6 видеороликов.

In [6]:
len(docs)

228

In [ ]:
segments[0]

{'id': 0,
 'seek': 0,
 'start': 0.0,
 'end': 2.8,
 'text': ' Добрый день, с вами Сергей Трощенков.',
 'tokens': [50381,
  3401,
  13829,
  4851,
  13509,
  11,
  776,
  24166,
  38393,
  2345,
  3200,
  9938,
  2000,
  1008,
  7718,
  13,
  50512],
 'temperature': 0.0,
 'avg_logprob': -0.13554080734905014,
 'compression_ratio': 1.8066465256797584,
 'no_speech_prob': 0.005384111776947975,
 'words': [{'word': ' Добрый',
   'start': 0.0,
   'end': 1.06,
   'probability': 0.9641097585360209},
  {'word': ' день,',
   'start': 1.06,
   'end': 1.28,
   'probability': 0.9999740123748779},
  {'word': ' с', 'start': 1.32, 'end': 1.68, 'probability': 0.999729335308075},
  {'word': ' вами',
   'start': 1.68,
   'end': 1.84,
   'probability': 0.9995624423027039},
  {'word': ' Сергей',
   'start': 1.84,
   'end': 2.28,
   'probability': 0.9999774694442749},
  {'word': ' Трощенков.',
   'start': 2.28,
   'end': 2.8,
   'probability': 0.968944501876831}]}

In [7]:
for doc in docs:
  print(doc.metadata)

{'link': 'https://www.youtube.com/embed/DfT_xXDQyCQ', 'time': 0}
{'link': 'https://www.youtube.com/embed/DfT_xXDQyCQ', 'time': 82}
{'link': 'https://www.youtube.com/embed/DfT_xXDQyCQ', 'time': 164}
{'link': 'https://www.youtube.com/embed/DfT_xXDQyCQ', 'time': 240}
{'link': 'https://www.youtube.com/embed/DfT_xXDQyCQ', 'time': 328}
{'link': 'https://www.youtube.com/embed/DfT_xXDQyCQ', 'time': 407}
{'link': 'https://www.youtube.com/embed/DfT_xXDQyCQ', 'time': 499}
{'link': 'https://www.youtube.com/embed/DfT_xXDQyCQ', 'time': 614}
{'link': 'https://www.youtube.com/embed/DfT_xXDQyCQ', 'time': 713}
{'link': 'https://www.youtube.com/embed/DfT_xXDQyCQ', 'time': 810}
{'link': 'https://www.youtube.com/embed/DfT_xXDQyCQ', 'time': 907}
{'link': 'https://www.youtube.com/embed/DfT_xXDQyCQ', 'time': 1009}
{'link': 'https://www.youtube.com/embed/DfT_xXDQyCQ', 'time': 1089}
{'link': 'https://www.youtube.com/embed/DfT_xXDQyCQ', 'time': 1162}
{'link': 'https://www.youtube.com/embed/DfT_xXDQyCQ', 'time': 

### Векторизация фрагментов и создание векторного хранилища

На этом шаге код почти полностью повторяет примеры, которые мы делали в двух предыдущих туториалах. Единственное отличие заключается в том, что векторную базу мы будем сохранять локально, а не хранить в оперативной памяти. Это позволит нам использовать базу повторно и на другом компьютере.

In [8]:
%%time

from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs,
                                  encode_kwargs=encode_kwargs)

vector_store = FAISS.from_documents(
    docs,
    embedding=embedding,
)

vector_store.save_local("faiss_index")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

CPU times: user 1min 36s, sys: 12.3 s, total: 1min 48s
Wall time: 1min 59s


### Подготовка генеративной части пайплайна

Подключимся к GigaChat API.

In [9]:
from google.colab import userdata
auth = userdata.get('SBER_AUTH')

In [10]:
from langchain.chat_models.gigachat import GigaChat

llm = GigaChat(credentials=auth, verify_ssl_certs=False, profanity_check=False)

### Подготовка промпта для RAG
Мы используем промпт, который уже был в прошлых туториалах.

В промпте сказано, что если в контексте не найдется ответа, LLM должна об этом сообщить. Но при этом у нас по-прежнему будет контекст из фрагментов, а значит пользователю будут показаны видео, несмотря на то, что LLM считает, что в них нет ответа на поставленный вопрос. Это недостаток предлагаемой системы.



In [11]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template('''Ответь на вопрос пользователя. \
Используй при этом только информацию из контекста. Если в контексте нет \
информации для ответа, сообщи об этом пользователю.
Контекст: {context}
Вопрос: {input}
Ответ:'''
)

### Завершение сборки пайплайна



In [12]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

embedding_retriever = vector_store.as_retriever(search_kwargs={"k": 3})

document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
    )

retrieval_chain = create_retrieval_chain(embedding_retriever, document_chain)

In [13]:
retrieval_chain.invoke({"input": 'Как создать агента?'})

{'input': 'Как создать агента?',
 'context': [Document(page_content=' описание вот этого агента, преобразует вот все инструменты и, значит, нашу языковую модель преобразует в структурированную систему с промтом, из которого понятно языковой модели, какие инструменты у нее есть. Ну и дальше, собственно, чтобы активировать, как бы, реализовать этого агента,  мы должны использовать AgentExecutor класс, куда передаем вот это уже описание агента,  опять передаем инструменты, собственно.  Ну и я еще рекомендую поставить,    вербоу стру, для того, чтобы мы, по крайней мере, на этапе  отладки нашего агента могли видеть, как бы, подноготную,  к каким функциям он обращается, какие там внутри процессы происходят.  Это довольно-таки, довольно-таки будет нам полезно.  Ну и история чата, пусть у нас будет пустая.  Тут есть некоторые различия между примерами и документацией,  которые идут в лонгчейне, и то, что предоставлено в гигачейне.  Дело в том, что, как правило, ну, во-первых, там своя функция 

In [ ]:
vector_store.similarity_search('Зачем нужно системное сообщение?')

[Document(page_content=' для демонстрации перед некоторой аудиторией школьников, я для подстраховки сделал здесь небольшую задержку. Дело в том, что в freemium версии нашего гигачата там многопоточность никакая не подразумевается,  насколько я понимаю.  То есть можно только одно сообщение отправлять за раз  и до тех пор, пока не придет ответ, следующее отправлять нельзя.  Вот чтобы там не произошло каких-нибудь накладок,  добавлена вот эта вот задержка.  Ну, благо, что гигачат работает довольно быстро  и никаких проблем с этим не возникает.  Я пробовал вот этот пример на аудитории,  там порядка... аудитория была побольше,  но реально писала сообщение порядка 10 школьников,  и в принципе этот программный код с таким объемом справлялся.  То есть ну нужно, конечно, оговориться, что это пример именно такой учебный,  отладочный и так далее.  Это, конечно, нельзя использовать как полноценного бота для работы 24 на 7 под большой нагрузкой.  Но, тем не менее, для  Это мы просто объявили вот эт

### Скачивание векторной базы

Теперь, когда мы убедились, что все работает, можно скачать БД, чтобы далее развернуть приложение на стороннем сервере.

In [14]:
import shutil

# Укажите путь к папке, которую хотите архивировать
folder_path = '/content/faiss_index'

# Укажите путь и имя для ZIP-архива
zip_path = 'faiss_index.zip'

# Создание ZIP-архива
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', folder_path)


'/content/faiss_index.zip'

In [15]:
from google.colab import files

# Скачивание ZIP-архива
files.download(zip_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
db.similarity_search('Зачем нужно системное сообщение?')

[Document(page_content=' для демонстрации перед некоторой аудиторией школьников, я для подстраховки сделал здесь небольшую задержку. Дело в том, что в freemium версии нашего гигачата там многопоточность никакая не подразумевается,  насколько я понимаю.  То есть можно только одно сообщение отправлять за раз  и до тех пор, пока не придет ответ, следующее отправлять нельзя.  Вот чтобы там не произошло каких-нибудь накладок,  добавлена вот эта вот задержка.  Ну, благо, что гигачат работает довольно быстро  и никаких проблем с этим не возникает.  Я пробовал вот этот пример на аудитории,  там порядка... аудитория была побольше,  но реально писала сообщение порядка 10 школьников,  и в принципе этот программный код с таким объемом справлялся.  То есть ну нужно, конечно, оговориться, что это пример именно такой учебный,  отладочный и так далее.  Это, конечно, нельзя использовать как полноценного бота для работы 24 на 7 под большой нагрузкой.  Но, тем не менее, для  Это мы просто объявили вот эт